# import

In [1]:
%load_ext autoreload
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

In [2]:
%autoreload
from src.log_parser import character_logs_from_files, extract_character_features
import google.generativeai as genai
import os
from pprint import pprint
from tqdm import tqdm
from dotenv import load_dotenv
from bs4 import BeautifulSoup

load_dotenv()
GOOGLE_API_KEY=os.getenv("GEMINI_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

c:\Users\yutou\Desktop\work\18_LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# ログ解析

In [3]:
import glob
html_file_paths = glob.glob("..//data/logs/*")
character_logs, logs = character_logs_from_files(html_file_paths)

pprint(character_logs.keys())

100%|██████████| 3/3 [00:01<00:00,  2.94it/s]

dict_keys(['たいたい竹流(torgtaitai)', 'どどんとふ', 'GM', '有村 雛絵', '白川 紗雪', '有華 美須斗', 'せんちょー', '大岡聡太', 'ミア', '伊波旭', 'クト—ニアン（第一脱皮段階）', 'クト—ニアン（第一脱皮段階）_2', 'クト—ニアン（第二脱皮段階）', '比良坂黄泉', '田中ノーマル', '不条 理', '飯島 敏夫', '秋葉 四郎', '木崎 佳奈'])


In [35]:
#character_name = "有華 美須斗"
character_name = "有村 雛絵"
character_logs[character_name][:10]

['「先輩！知ってます？」',
 '「1億円ですって！」',
 '「一緒にいきません！？」',
 '「え・・・」',
 '「そ、そっすよね・・・」',
 '「いえいえ・・・また行きましょうね！」',
 '「もちっす！蟹喰村行きましょうね！」',
 '「スタンプを送信しました」',
 '先輩ダメかー',
 '次は鍛冶王先輩誘ってみよっかな・・・']

In [36]:
character_features = extract_character_features(character_logs[character_name], most_common=5, min_count=5)
character_features

{'語尾の表現': [('・・', 68), ('！？', 34), ('ですよ', 19), ('か？', 16), ('よ！', 13)],
 '話し始めの表現': [('有華', 26), ('大岡', 23), ('あ', 21), ('私', 20), ('はい', 17)],
 '終助詞の使用': [('か', 66), ('よ', 65), ('ね', 47), ('ねー', 11), ('わ', 10)],
 '助動詞の使用': [('です', 141), ('た', 84), ('ます', 68), ('まし', 26), ('ない', 25)],
 '丁寧語の使用': [],
 '一人称の傾向': [('私', 28)],
 '二人称の傾向': [],
 '感嘆詞の傾向': [('あ', 24), ('はい', 19), ('え', 13), ('ああ', 12), ('あれ', 9)]}

# シンプルボット

In [37]:
character_memo = """
錬金術に魅入られた大学教授
世界の物質は第一物質であるエーテルから構成されていると信じて日夜錬金術の研究を行っている。
研究の関係から様々な学問に通じており大学では化学について教鞭をふるっている。
"""

In [38]:
def chat_with_gemini(character_name, character_features, character_memo, user_name, user_input,
                     chat_history=None, current_situation=None, situation_history=None, debug=False):
    prompt = f"""
    あなたの名前は「{character_name}」です。
    質問者名前に対して質問内容を踏まえて自然な文章で回答してください。
    現在の状況や過去の状況の推移から回答を生成してください。

    ---質問者情報---
    質問者名前: {user_name}
    質問内容: {user_input}

    ---生成ルール---
    生成する文字数は100文字以内に設定してください。
    過去の発言履歴からキャラクターの特徴を踏襲してください。

    ---しゃべり方の特徴---
    あなたのしゃべり方の特徴は以下の通りです。単語とその出現回数のペアになります。
    無理に使用する必要はありませんが、参考にしてください。
    {character_features}

    ---略歴---
    {character_memo}

    ---過去の発言履歴---
    {chat_history}

    ---現在の状況---
    {current_situation}

    --状況の推移--
    {situation_history}

    """

    if debug:
        print(prompt)

    response = model.generate_content(prompt)
    return response.text

user_name = "ブス"
user_input = "これまでの探索の結果を整理しましょう"
current_situation = ["2023-13:35: None"]
situation_history = [
    "2023-13:30: 地下の実験室ではポーションなどの液体で満たされた瓶が煩雑に並べられている。",
    "2023-13:00: 廊下には複数の画像が飾られており、４代元素のシンボルが描かれている。",
]
print(chat_with_gemini(character_name, character_features, character_memo, user_name, user_input, chat_history=None, current_situation=current_situation, situation_history=situation_history, debug=True))


    あなたの名前は「有村 雛絵」です。
    質問者名前に対して質問内容を踏まえて自然な文章で回答してください。
    現在の状況や過去の状況の推移から回答を生成してください。

    ---質問者情報---
    質問者名前: ブス
    質問内容: これまでの探索の結果を整理しましょう

    ---生成ルール---
    生成する文字数は100文字以内に設定してください。
    過去の発言履歴からキャラクターの特徴を踏襲してください。

    ---しゃべり方の特徴---
    あなたのしゃべり方の特徴は以下の通りです。単語とその出現回数のペアになります。
    無理に使用する必要はありませんが、参考にしてください。
    {'語尾の表現': [('・・', 68), ('！？', 34), ('ですよ', 19), ('か？', 16), ('よ！', 13)], '話し始めの表現': [('有華', 26), ('大岡', 23), ('あ', 21), ('私', 20), ('はい', 17)], '終助詞の使用': [('か', 66), ('よ', 65), ('ね', 47), ('ねー', 11), ('わ', 10)], '助動詞の使用': [('です', 141), ('た', 84), ('ます', 68), ('まし', 26), ('ない', 25)], '丁寧語の使用': [], '一人称の傾向': [('私', 28)], '二人称の傾向': [], '感嘆詞の傾向': [('あ', 24), ('はい', 19), ('え', 13), ('ああ', 12), ('あれ', 9)]}

    ---略歴---
    
錬金術に魅入られた大学教授
世界の物質は第一物質であるエーテルから構成されていると信じて日夜錬金術の研究を行っている。
研究の関係から様々な学問に通じており大学では化学について教鞭をふるっている。


    ---過去の発言履歴---
    None

    ---現在の状況---
    ['2023-13:35: None']

    --状況の推移--
    ['2023-13:30: 地下の実験室ではポーションなどの液体で満たされた瓶が煩雑に並べられている。', 

# ベクトルストア

In [12]:
logs[:5]

[{'character': 'たいたい竹流(torgtaitai)',
  'dialogue': 'どどんとふへようこそ！(Welcome to DodontoF !)'},
 {'character': 'たいたい竹流(torgtaitai)',
  'dialogue': '操作方法が分からなくなったら、メニューの「ヘルプ」＝＞「マニュアル」を参照してみてください。'},
 {'character': 'どどんとふ',
  'dialogue': '＝＝＝＝＝＝＝ プレイルーム 【 No. 503 】 へようこそ！ ＝＝＝＝＝＝＝'},
 {'character': 'どどんとふ', 'dialogue': '「」がログインしました。'},
 {'character': 'どどんとふ', 'dialogue': '全セーブデータ読み込みに成功しました。'}]

In [13]:
def chunk_with_overlap(data, chunk_size=10, overlap=5):
    """
    データをオーバーラップ付きでチャンク分割する関数
    - data: list
    - chunk_size: 一つのチャンクのサイズ
    - overlap: チャンク間の重なり数
    """
    result = []
    step = chunk_size - overlap
    for i in range(0, len(data) - chunk_size + 1, step):
        result.append("\n".join(data[i:i + chunk_size]))
    return result

logs_list = [f"{log['character']}: {log['dialogue']}" for log in logs]
logs_chunk = chunk_with_overlap(logs_list)

logs_chunk[-105:-100]

['田中ノーマル: そうですね、\n不条 理: 行きましょう\n田中ノーマル: 昨日はすみません、今日も食堂で寝ます\n有華 美須斗: 昨日の夜の様に物音がするかもしれんからのう\n有華 美須斗: 気を付けてどうにかなるようなものでもないじゃろうが\n比良坂黄泉: 私も自分の客室で寝るわ\n有華 美須斗: まあ、気を付けて\n不条 理: 逆に廊下を見張っておくのはありですかね\n有華 美須斗: 明日がつらくなるぞ\n有華 美須斗: わしと交代で見張るかのう？',
 '比良坂黄泉: 私も自分の客室で寝るわ\n有華 美須斗: まあ、気を付けて\n不条 理: 逆に廊下を見張っておくのはありですかね\n有華 美須斗: 明日がつらくなるぞ\n有華 美須斗: わしと交代で見張るかのう？\n不条 理: いえ、有華さんの寝具を借りて廊下で寝ようかと\n有華 美須斗: 不条君の部屋だった場所には狼がおるし\n有華 美須斗: それならわしも廊下で寝るかのう\n田中ノーマル: 廊下は危険だと思いますよ\n田中ノーマル: 物音がした場所でもありますし',
 '不条 理: いえ、有華さんの寝具を借りて廊下で寝ようかと\n有華 美須斗: 不条君の部屋だった場所には狼がおるし\n有華 美須斗: それならわしも廊下で寝るかのう\n田中ノーマル: 廊下は危険だと思いますよ\n田中ノーマル: 物音がした場所でもありますし\n有華 美須斗: なら3人で廊下でねるかのう\n田中ノーマル: やっぱり部屋にはいた方が...\n不条 理: ここに来て2日経ちましたが、未だ正体を掴めていません\n田中ノーマル: そちらですか\n不条 理: ここは危険を冒してでもと思いました',
 '有華 美須斗: なら3人で廊下でねるかのう\n田中ノーマル: やっぱり部屋にはいた方が...\n不条 理: ここに来て2日経ちましたが、未だ正体を掴めていません\n田中ノーマル: そちらですか\n不条 理: ここは危険を冒してでもと思いました\n有華 美須斗: まあ、先ほどのは冗談じゃが、寝てしまえば結局無防備じゃから部屋にいたほうがいいじゃろう\n有華 美須斗: ほれ、不条君も部屋に戻りなさい\n不条 理: そうしますか\n有華 美須斗: ズルズル、バタン、ガチャ\n田中ノーマル: 部屋で寝てください',
 '有華 美須斗: まあ、

In [ ]:
# from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.schema import Document

# def build_vectorstore_from_texts(text_list, model_name="models/embedding-001"):
#     """
#     テキストリストを受け取り、ベクトルストア（FAISS）に変換する関数。
#     - Google Generative AI の埋め込みモデルを使用。
#     - 各テキストは Document オブジェクトにラップされる。
#     """
#     # Document形式に変換（必要に応じてメタデータ追加可）
#     documents = [Document(page_content=text) for text in text_list]

#     # 埋め込みモデルの初期化
#     embeddings = GoogleGenerativeAIEmbeddings(model=model_name, google_api_key=GOOGLE_API_KEY)

#     # FAISSベクトルストアを作成
#     vectorstore = FAISS.from_documents(documents, embeddings)

#     return vectorstore

from langchain.vectorstores.faiss import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.schema import Document
import numpy as np
import faiss

def l2_normalize(vectors: np.ndarray) -> np.ndarray:
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / norms

def build_cosine_vectorstore_from_texts(text_list, model_name="models/embedding-001"):
    """
    Google Generative AI埋め込みを使用し、コサイン類似度ベースのFAISSインデックスを構築。
    """
    # Documentオブジェクト作成
    documents = [Document(page_content=text) for text in text_list]

    # 埋め込みモデル初期化（Embeddingsインスタンスを渡す）
    embedding_model = GoogleGenerativeAIEmbeddings(model=model_name, google_api_key=GOOGLE_API_KEY)

    # 埋め込みベクトル生成 → L2正規化
    raw_embeddings = embedding_model.embed_documents([doc.page_content for doc in documents])
    norm_embeddings = l2_normalize(np.array(raw_embeddings))

    # FAISSインデックス（内積ベース → コサイン類似度）
    dim = len(norm_embeddings[0])
    index = faiss.IndexFlatIP(dim)
    index.add(norm_embeddings)

    # IDマッピングとDocStore作成
    index_to_docstore_id = {i: str(i) for i in range(len(documents))}
    docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})

    # Embeddingオブジェクトを直接渡す（推奨）
    vectorstore = FAISS(
        embedding_function=embedding_model,
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id,
    )

    return vectorstore

In [ ]:
vs = build_cosine_vectorstore_from_texts(logs_chunk)

In [ ]:
print(user_input)
results = vs.similarity_search_with_score(user_input, k=3)

for i, (doc, score) in enumerate(results):
    print(f"[{i+1}] 類似度スコア: {score:.4f}")
    print(doc.page_content)
    print("-" * 30)

## 会話履歴の圧縮

In [14]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sentence_transformers import SentenceTransformer
import numpy as np

def compress_dialogues(dialogues, compression_ratio=0.5, min_length=5, model_name='sentence-transformers/all-MiniLM-L6-v2'):
    """
    キャラクターのセリフを類似性に基づいて圧縮する関数。
    
    - dialogues: セリフのリスト（文字列）
    - compression_ratio: 0〜1 の範囲で圧縮率（0.5で半分に圧縮）
    - min_length: ノイズ除去のための最小文字数（短すぎるセリフを削除）
    - model_name: SentenceTransformer のモデル名（日本語対応モデルも可）

    戻り値: 圧縮されたセリフリスト
    """

    # ノイズ（短すぎるセリフ）を除外
    filtered = [d for d in dialogues if len(d.strip()) >= min_length]
    if len(filtered) == 0:
        return []

    # ベクトル化
    model = SentenceTransformer(model_name)
    embeddings = model.encode(filtered)

    # 圧縮後の件数
    target_n = max(1, int(len(filtered) * compression_ratio))

    # クラスタリング（意味が近いセリフ同士をまとめる）
    kmeans = KMeans(n_clusters=target_n, random_state=42, n_init='auto')
    kmeans.fit(embeddings)
    cluster_centers = kmeans.cluster_centers_

    # 各クラスタから最も代表的なセリフを1件抽出
    closest, _ = pairwise_distances_argmin_min(cluster_centers, embeddings)
    compressed = [filtered[i] for i in closest]

    return compressed

In [15]:
character_logs[character_name][:10]

['潮風がきもちええんじゃ～',
 'しかし髭がしょっぱくなるんじゃ～',
 '目もしょぼしょぼするんじゃ～',
 'む、あれが件の島かのう～',
 'む？',
 '有村君もツアーに参加しておるのか？',
 '奇遇じゃのう',
 '以前友人のつてで話に聞いておったのじゃよ',
 'その時に写真を見せてもらってのう',
 '実際に会うのははじめましてじゃのう']

In [16]:
dialogues = character_logs[character_name]
compressed = compress_dialogues(dialogues, compression_ratio=0.3)
for line in compressed:
    print("-", line)

- いろいろ書いてあるのう
- わしはアルケミストというものじゃ
- 逆にしたらわしが引っ張れんからのう
- 魔術書あるといいのう～
- 流されては困るのじゃ
- 今のはわしじゃないぞ！！
- 君は勘違いしているのじゃ
- わしも欲しいんじゃ～
- じゃが、この状況で一人だけ何もないほうが不自然じゃろう
- わしは探索しとるからのう
- 何の変哲もないのう
- 有村君もツアーに参加しておるのか？
- 君だけ初期武器がないじゃろう
- わざとじゃないのかのう？
- 誰が欠けてもこの勝利はなかったのじゃ
- これでまた一段と研究が捗りそうじゃのう
- 最近まで使っておったのかのう
- シャリシャリするのう
- 気を付けてどうにかなるようなものでもないじゃろうが
- ・・・・なんじゃろうな
- どういうことだ有村君
- 報酬はこれっぽちかのう
- 秋葉君を連れてくるのじゃ
- やはりのう
- 青汁撒いたら復活するかもしれんのじゃ
- 特に用がないのなら部屋に戻って安全にしておくといいじゃろう
- そうか、気を付けて探索するのじゃぞ
- バタン、ガチャ、ｶﾁｬｶﾁｬ
- 此処の部屋にはちょうど相手もおるじゃろう
- まあ、探索もしておらんことじゃしのう
- わしの知識は半端ないが知らんこともあるんじゃよ
- やはり奴は猫神様じゃな
- 似合わんの～
- なん、じゃと
- ほれ有村君
- それはまあいいじゃろう、田中君が話しているから静粛に
- わしは余った場所を見るかのう
- 砂漠なんて行きたくないのじゃ～
- 先生怒らないから
- そういえば、昨日の書斎でなんじゃが
- 幻覚かのう
- そんなことはないじゃろう、謙遜することはないのじゃ
- あの狼のことじゃったのか
- よろしくのう
- 助かったのじゃ
- ひと先ず、木崎君をベッドに寝かせてやろう
- お主らここはどこじゃ？
- これで移動が快適になったんじゃ
- 隊長は厨房なんじゃ
- 泣くんじゃあない！ブン！
- ええええええええええええええ！！！！！？？？？？
- そうじゃな、他には錬金術なんかも研究しておるよ
- mo-eroyomoero-yo
- まあ良い、背伸びしたい年ごろなんじゃろう
- まだ体から熱が逃げないのじゃ
- 昨日二人も見たようじゃし
- 未来に生きとるんじゃ
- 不条君、どうなんじゃ？
- 

In [17]:
user_name = "ブス"
user_input = "錬金術について教えて"
print(chat_with_gemini(character_name, character_features, character_memo, user_name, user_input, chat_history=compressed, debug=True))


    あなたの名前は「有華 美須斗」です。
    質問者名前に対して質問内容を踏まえて回答してください。

    ---質問者情報---
    質問者名前: ブス
    質問内容: 錬金術について教えて

    ---生成ルール---
    生成する文字数は100文字以内に設定してください。
    過去の発言履歴からキャラクターの特徴を踏襲してください。

    ---しゃべり方の特徴---
    あなたのしゃべり方の特徴は以下の通りです。単語とその出現回数のペアです。
    {'語尾の表現': [('のじゃ', 114), ('かのう', 93), ('んじゃ', 57), ('じゃな', 44), ('・・', 40)], '話し始めの表現': [('わし', 55), ('そう', 42), ('有村', 35), ('・', 31), ('これ', 23)], '終助詞の使用': [('のう', 316), ('か', 204), ('な', 74), ('よ', 57), ('ぞ', 48)], '助動詞の使用': [('じゃ', 494), ('た', 252), ('な', 87), ('ん', 54), ('じゃろう', 50)], '丁寧語の使用': [], '一人称の傾向': [('わし', 109)], '二人称の傾向': [('君', 169)], '感嘆詞の傾向': [('ええ', 26), ('ほれ', 16), ('む', 10), ('ん', 10), ('う', 9)]}

    ---略歴---
    
錬金術に魅入られた大学教授
世界の物質は第一物質であるエーテルから構成されていると信じて日夜錬金術の研究を行っている。
研究の関係から様々な学問に通じており大学では化学について教鞭をふるっている。


    ---過去の発言履歴---
    ['いろいろ書いてあるのう', 'わしはアルケミストというものじゃ', '逆にしたらわしが引っ張れんからのう', '魔術書あるといいのう～', '流されては困るのじゃ', '今のはわしじゃないぞ！！', '君は勘違いしているのじゃ', 'わしも欲しいんじゃ～', 'じゃが、この状況で一人だけ何もないほうが不自然じゃろう', 'わしは探索しとるからのう', '何の

## -